In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import pandas as pd
from sqlalchemy import create_engine

from config import pwd


In [3]:
import sys

In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
db = client.quotes_db
collections = db.scraped_quotes
collections_1 = db.result

In [6]:
scraped_quotes = []
for x in range(1, 11):
    try:
        url = 'https://quotes.toscrape.com/page/'
        response  = requests.get(url + str(x))
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        quotes = soup.find_all('div', class_ = 'quote')
        for q in range(0, len(quotes)):
            quote = quotes[q]
            quote_text = quote.text.strip().split('\n')[0]
            author = quote.find('small' , class_='author').text
            tags = [tag.text for tag in quote.find('div', class_='tags').find_all('a', class_='tag')]
            scraped_quotes.append({'quote' : quote_text, 'author' : author, 'tags' : tags})
    except Exception as e:
            print(e)

In [7]:
len(scraped_quotes)

100

In [8]:
result = []
for y in range(1, 11):
    try:
        url = 'https://quotes.toscrape.com/page/'
        response  = requests.get(url + str(y))
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        quotes = soup.find_all('div', class_ = 'quote')
        for w in range(0, len(quotes)):
            quote = quotes[w]
            link = quote.a['href']
            about_link = requests.get('https://quotes.toscrape.com/' + link)
            html_1 = about_link.text
            soup_1 = BeautifulSoup(html_1, 'html.parser')
            born_info = soup_1.find('span' , class_ = 'author-born-date').text.strip().split('\n')[0]
            name_info = soup_1.find('h3', class_ = 'author-title').text.strip().split('\n')[0]
            description_info = soup_1.find('div' , class_="author-description").text.strip().split('\n')[0]
            result.append({'name':name_info, 'born': born_info, 'description':description_info})
            
    except Exception as e:
        print(e)

In [9]:
collections

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'quotes_db'), 'scraped_quotes')

In [10]:
collections.insert_many(scraped_quotes)

In [11]:
collections_1.insert_many(result)

In [12]:
result_from_mongo = db.result
df1 = pd.DataFrame(list(result_from_mongo.find()))
df1 = df1[['name', 'born', 'description']]
df1

,name,born,description
0,Albert Einstein,"March 14, 1879","In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,"July 31, 1965",See also: Robert GalbraithAlthough she writes ...
2,Albert Einstein,"March 14, 1879","In 1879, Albert Einstein was born in Ulm, Germ..."
3,Jane Austen,"December 16, 1775",Jane Austen was an English novelist whose work...
4,Marilyn Monroe,"June 01, 1926",Marilyn Monroe (born Norma Jeane Mortenson; Ju...
...,...,...,...
495,Harper Lee,"April 28, 1926","Harper Lee, known as Nelle, was born in the Al..."
496,Madeleine L'Engle,"November 29, 1918",Madeleine L'Engle was an American writer best ...
497,Mark Twain,"November 30, 1835","Samuel Langhorne Clemens, better known by his ..."
498,Dr. Seuss,"March 02, 1904",Theodor Seuss Geisel was born 2 March 1904 in ...


In [13]:
scraped_quotes_from_mongo = db.scraped_quotes
df2 = pd.DataFrame(list(scraped_quotes_from_mongo.find()))
df2 = df2[['quote', 'author', 'tags']]
df2

,quote,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
495,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
496,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
497,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
498,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


In [17]:
quote_df = df2[['quote', 'author','tags']]

In [18]:
quote_df = quote_df.rename(columns = {'id' : 'quote_id', 'quote' : 'quote_text', 'author':'author_name', 'tags':'tag'})

In [19]:
author_df = df1[['name', 'born', 'description']]
author_df = author_df.drop_duplicates()


In [20]:
quote_df.insert(0,'id',quote_df.index + 1)

In [21]:
author_df.insert(0,'id',author_df.index + 1)

In [22]:
quote_df = quote_df.rename(columns = {'id' : 'quote_id'})
quote_df

,quote_id,quote_text,author_name,tag
0,1,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,2,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,3,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,4,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,5,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...,...
495,496,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
496,497,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
497,498,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
498,499,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


In [23]:
author_df = author_df.rename(columns = {'id' : 'author_id'})

In [25]:
tags = quote_df[['quote_id', 'tag']]
tags

,quote_id,tag
0,1,"[change, deep-thoughts, thinking, world]"
1,2,"[abilities, choices]"
2,3,"[inspirational, life, live, miracle, miracles]"
3,4,"[aliteracy, books, classic, humor]"
4,5,"[be-yourself, inspirational]"
...,...,...
495,496,[better-life-empathy]
496,497,"[books, children, difficult, grown-ups, write,..."
497,498,[truth]
498,499,[inspirational]


In [26]:
quote_df = quote_df.drop('tag', axis=1)
quote_df

,quote_id,quote_text,author_name
0,1,“The world as we have created it is a process ...,Albert Einstein
1,2,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,3,“There are only two ways to live your life. On...,Albert Einstein
3,4,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,5,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
...,...,...,...
495,496,“You never really understand a person until yo...,Harper Lee
496,497,“You have to write the book that wants to be w...,Madeleine L'Engle
497,498,“Never tell the truth to people who are not wo...,Mark Twain
498,499,"“A person's a person, no matter how small.”",Dr. Seuss


In [37]:
rds_connection_string = f"postgres:{pwd}@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['author', 'quote', 'tag']

In [30]:
author_df.to_sql(name='author', con=engine, if_exists='append', index=False)

In [31]:
quote_df.to_sql(name='quote', con=engine, if_exists='append', index=False)

In [32]:
tags['tag'][99]

['books', 'mind']

In [33]:
tags = tags.explode('tag')

In [34]:
sys.setrecursionlimit(100000)

In [36]:
tags.to_sql(name='tag', con=engine, if_exists='append', index=False)